In [1]:
# main_folder = '/home/pgajo/working/incels/ECIR/results/ECIR_substitution_fixed_testing'
# main_folder = '/home/pgajo/working/incels/ECIR/results/ECIR_substitution_fixed_training'
# main_folder = '/home/pgajo/working/incels/ECIR/results/ECIR_union_fixed_training'
main_folder = '/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_substitution_fixed_testing'
# main_folder = '/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_substitution_fixed_training'
# main_folder = '/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_union_fixed_training'

padding_right_modifier = 10
padding_left_modifier = 8
legend_height = 1.4
import os
import glob
import pandas as pd

# Create an empty list to store DataFrames
dfs = []
# Iterate over all subfolders in the main folder
for root, dirs, files in os.walk(main_folder):
    # Iterate over all CSV files in the subfolder
    for file in glob.glob(os.path.join(root, '*.csv')):
        # Check if the filename starts with 'ECIR'
        if os.path.basename(file).startswith('ECIR'):
            # Read the CSV file into a DataFrame
            temp_df = pd.read_csv(file)
            temp_df['dir'] = os.path.dirname(file)
            # print('temp_df', temp_df)
            # Add the DataFrame to the list
            dfs.append(temp_df)

# drop model column
for df in dfs:
    df.drop(columns=['model', 'train_set(s)', 'dev_set(s)', 'test_set(s)'], inplace=True)

# within column 'train_source', sub IFD-EN-5203_truncated_0.tsv with 0
for df in dfs:
    df['train_source'] = df['train_source'].str.replace('IFD-EN-5203_truncated_', '')
    df['train_source'] = df['train_source'].str.replace('.tsv', '')
    df['dev_source'] = df['dev_source'].str.replace('IFD-EN-5203_truncated_', '')
    df['dev_source'] = df['dev_source'].str.replace('.tsv', '')
    df['test_source'] = df['test_source'].str.replace('IFD-EN-5203_truncated_', '')
    df['test_source'] = df['test_source'].str.replace('.tsv', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results/ECIR_substitution_fixed_testing', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results/ECIR_substitution_fixed_training', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results/ECIR_union_fixed_training', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_substitution_fixed_testing', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_substitution_fixed_training', '')
    df['dir'] = df['dir'].str.replace('/home/pgajo/working/incels/ECIR/results_ECIR_fix/ECIR_union_fixed_training', '')
print('len(dfs)', len(dfs))
import os
import pandas as pd

# Step 1: Concatenate all individual dataframes into one
concatenated_df = pd.concat(dfs, ignore_index=True)
# Step 2: Group by 'dir' and 'epoch', then compute the average 'test_f1' for each group
grouped_df = concatenated_df.groupby(['dir', 'epoch'])['test_f1'].mean().reset_index()
# get st.dev
grouped_df['test_f1_stdev'] = concatenated_df.groupby(['dir', 'epoch'])['test_f1'].std().reset_index()['test_f1']

# Step 3 & 4: Find the epoch with the highest average 'test_f1' for each directory
# and create a new DataFrame to store these results
final_results = pd.DataFrame()

for dir_name, group in grouped_df.groupby('dir'):
    max_row = pd.DataFrame(group.loc[group['test_f1'].idxmax()])
    final_results = pd.concat([final_results, max_row.T])

# Reset index for better readability
final_results.reset_index(drop=True, inplace=True)

# save to csv
display(final_results)
final_results.to_csv(os.path.join(main_folder, 'final_results.csv'), index=False)

len(dfs) 5


,dir,epoch,test_f1,test_f1_stdev
0,/metrics_truncated_0/full/incel-bert-base-unca...,4.0,0.870059,0.006168


In [2]:
# Reset index for better readability
grouped_df.reset_index(drop=True, inplace=True)

# Display the DataFrame
display(grouped_df)

,dir,epoch,test_f1,test_f1_stdev
0,/metrics_truncated_0/full/incel-bert-base-unca...,1.0,0.850411,0.004604
1,/metrics_truncated_0/full/incel-bert-base-unca...,2.0,0.851267,0.006606
2,/metrics_truncated_0/full/incel-bert-base-unca...,3.0,0.866067,0.008300
3,/metrics_truncated_0/full/incel-bert-base-unca...,4.0,0.870059,0.006168
